In [5]:
import os
directory = f'/Users/lilimatic/greenspace'
os.chdir(directory)

from sklearn.ensemble import RandomForestClassifier


from sklearn.model_selection import train_test_split, cross_val_score  
from sklearn.utils.class_weight import compute_sample_weight

from sklearn.metrics import roc_curve, roc_auc_score, classification_report, confusion_matrix, accuracy_score 

import matplotlib.pyplot as plt


In [6]:
from functions.ml_data_hot_one import *

df_copy = df.copy()

In [ ]:
df.plot(x='safety',y=df.safety,kind='bar')

In [ ]:
sum(df.safety)/len(df.safety)

In [ ]:
y = 'safety'

X_train, X_test, y_train, y_test = train_test_split(df_copy.loc[:, df_copy.columns != y], \
                                                    df_copy[y], test_size=0.25)

In [ ]:
#Similar proportions
sum(y_test)/len(y_test)

In [ ]:
sample_weights = compute_sample_weight(
    class_weight='balanced',
    y=y_train #provide your own target name
)


rf = RandomForestClassifier(n_estimators=15,max_depth=8,random_state=24,criterion='gini',min_samples_split=10)

In [ ]:
#Model fit 

rf.fit(X_train, y_train)


In [ ]:
print('Training accuracy: ', np.mean(rf.predict(X_train) ==y_train)*100) 
print ('Test accuracy:', np.mean(rf.predict(X_test) ==y_test)*100)

In [ ]:
#plot_confusion_matrix(model,X_test,y_test,values_format='d',display_labels=['Safe','Unsafe'])

In [ ]:
pred_prob3 = rf.predict_proba(X_test)

In [ ]:
fpr3, tpr3, thresh3 = roc_curve(y_test, pred_prob3[:,1], pos_label=1)

In [ ]:
random_probs = [0 for i in range(len(y_test))]
p_fpr, p_tpr, _ = roc_curve(y_test, random_probs, pos_label=1)

In [ ]:
# auc scores

auc_score3 = roc_auc_score(y_test, pred_prob3[:,1])

print(auc_score3)

In [ ]:
plt.style.use('seaborn')

# plot roc curves
#plt.plot(fpr1, tpr1, linestyle='--',color='orange', label='XGB')
plt.plot(fpr3, tpr3, linestyle='--',color='purple') #label='Random Forest'
plt.plot(p_fpr, p_tpr, linestyle='--', color='blue')

#plt.title('ROC curve')
# x label
plt.xlabel('False Positive Rate')
# y label
plt.ylabel('True Positive rate')
plt.grid(False)

#ax = plt.axes()
#ax.set_facecolor("white")

plt.legend(loc='best')
plt.savefig('ROC',dpi=300)
plt.show();

In [ ]:
rf.score(X_test,y_test)

In [ ]:
y_pred = rf.predict(X_test)

In [ ]:
cm = confusion_matrix(y_test,y_pred)

cm

In [ ]:
rf.feature_importances_

In [ ]:
accuracy_score(y_pred ,y_test)

In [ ]:
cross_val_score(rf, X_train,y_train,cv=5)

In [ ]:

print(classification_report(y_pred, y_test))

In [ ]:
features = df.columns

importances = rf.feature_importances_
indices = np.argsort(importances)

values = list(rf.feature_importances_[indices])

names = [features[i] for i in indices]

importance = pd.DataFrame({'district' : names, 'importance': values})

importance = importance.sort_values('district')

In [ ]:
#merged will be summed 
merged = [] 
names_merged=[] 
for o in oldcolumns:
    if (len([col for col in df if col.startswith(o+'_')])>0):
        merged.append([col for col in df if col.startswith(o+'_')])
        names_merged.append(o)


sums_merged = []

for m in merged:
    #print(m)
    sums_merged.append(importance.loc[importance['district'].isin(m)].sum()[1])


In [ ]:
names_merged.extend(['municipalities','activity','socializing','age','time'])

In [ ]:
for x in [list_municipalities,list_activities,list_socializing]: 
    sums_merged.append(importance.loc[importance['district'].isin(list(x.columns))].sum()[1])

In [ ]:
sums_merged.extend([importance.loc[importance['district'].isin(['age'])]['importance'].iloc[0],
importance.loc[importance['district'].isin(['time'])]['importance'].iloc[0]])

In [ ]:
#remove effect of safety variable
sums_merged= sums_merged/sum(sums_merged)

In [ ]:
#importance.loc[importance['district'].isin(['safety'])]['importance'].iloc[0]

sum(sums_merged)

In [ ]:
important = pd.DataFrame({'variable': names_merged,'importance':sums_merged })

important = important.sort_values('importance')

In [ ]:
#important.plot.bar(x='variable',y='importance',rot=0)

plt.barh(important.variable,important.importance,color='red')
plt.grid(False)
plt.xlabel('importance')
plt.ylabel('variable')
plt.axes()
ax.set_facecolor('white')


In [ ]:
len(names_merged)

In [ ]:
for i in range(3):
    tree = rf.estimators_[i]
    dot_data = export_graphviz(tree,
                               feature_names=X_train.columns,  
                               filled=True,  
                               max_depth=2, 
                               impurity=False, 
                               proportion=True)
    graph = graphviz.Source(dot_data)
    display(graph)